# Amplifierd API - Module Management

This notebook demonstrates module discovery operations in the flat directory structure.

## Overview

Modules in amplifier provide core functionality:
- **Providers**: LLM service integrations (OpenAI, Anthropic, etc.)
- **Tools**: Capabilities available to LLMs (file ops, web search, etc.)
- **Hooks**: Event-driven behaviors (logging, notifications, etc.)
- **Orchestrators**: Coordination and workflow management

**Flat Directory Structure**:
```
{AMPLIFIERD_ROOT}/local/share/
└── modules/{collection}/{name}/
    └── module.yaml
```

**Module Discovery**: Scans `modules/**/*.yaml` recursively

**Module ID Format**: `{collection}/{name}` (e.g., `my-collection/openai`)

**Namespace Isolation**: Collection name prevents conflicts between packages

**Metadata**: Read from `module.yaml` files in module directories

This notebook covers **discovery operations**.

In [1]:
import json

import requests

BASE_URL = "http://127.0.0.1:8420"
API_BASE = f"{BASE_URL}/api/v1"


def print_response(response: requests.Response, title: str = "") -> None:
    if title:
        print(f"\n{'=' * 60}")
        print(f"{title}")
        print(f"{'=' * 60}")
    print(f"Status: {response.status_code} {response.reason}")
    if response.content:
        try:
            data = response.json()
            print(json.dumps(data, indent=2))
            return data
        except json.JSONDecodeError:
            print(response.text)
            return None
    return None


print("✓ Setup complete")

✓ Setup complete


## Module Discovery

### List All Modules

Get all available modules across all types:

In [4]:
response = requests.get(f"{API_BASE}/modules/")
modules = print_response(response, "LIST ALL MODULES")

if modules:
    print(f"\n✓ Found {len(modules)} module(s)")

    # Group by type
    by_type = {}
    for module in modules:
        module_type = module.get("type", "unknown")
        if module_type not in by_type:
            by_type[module_type] = []
        by_type[module_type].append(module)

    print("\nModules by type:")
    for module_type, modules_list in by_type.items():
        print(f"  - {module_type}: {len(modules_list)}")


LIST ALL MODULES
Status: 200 OK
[]


### List Modules by Type

Get modules filtered by specific type:

In [5]:
# List all providers
response = requests.get(f"{API_BASE}/modules/providers")
providers = print_response(response, "LIST PROVIDERS")

if providers:
    print(f"\n✓ Found {len(providers)} provider(s)")
    for provider in providers[:5]:  # Show first 5
        collection = provider.get("collection", "N/A")
        print(f"  - {provider['id']} (collection: {collection})")


LIST PROVIDERS
Status: 200 OK
[]


In [6]:
# List all tools
response = requests.get(f"{API_BASE}/modules/tools")
tools = print_response(response, "LIST TOOLS")

if tools:
    print(f"\n✓ Found {len(tools)} tool(s)")


LIST TOOLS
Status: 200 OK
[]


In [7]:
# List all hooks
response = requests.get(f"{API_BASE}/modules/hooks")
hooks = print_response(response, "LIST HOOKS")

if hooks:
    print(f"\n✓ Found {len(hooks)} hook(s)")


LIST HOOKS
Status: 200 OK
[]


In [8]:
# List all orchestrators
response = requests.get(f"{API_BASE}/modules/orchestrators")
orchestrators = print_response(response, "LIST ORCHESTRATORS")

if orchestrators:
    print(f"\n✓ Found {len(orchestrators)} orchestrator(s)")


LIST ORCHESTRATORS
Status: 200 OK
[]


### Get Module Details

Retrieve detailed information about a specific module:

In [9]:
if providers:
    module_id = providers[0]["id"]

    response = requests.get(f"{API_BASE}/modules/{module_id}")
    details = print_response(response, f"GET MODULE: {module_id}")

    if details:
        print(f"\n✓ Module: {details['id']}")
        print(f"  Type: {details['type']}")
        print(f"  Location: {details['location']}")
        if details.get("collection"):
            print(f"  Collection: {details['collection']}")

### Filter Modules by Type

Use query parameters to filter modules:

In [10]:
# Filter for providers only
response = requests.get(f"{API_BASE}/modules/?type=provider")
providers = print_response(response, "FILTER: type=provider")

if providers:
    print(f"\n✓ Found {len(providers)} provider module(s)")


FILTER: type=provider
Status: 200 OK
[]


### Understanding Module Structure

Each module has a specific directory structure and metadata format.

In [ ]:
# Module structure example
print("Module Directory Structure (Flat):")
print("=" * 60)
print("\n{AMPLIFIERD_ROOT}/local/share/")
print("└── modules/")
print("    └── my-collection/")
print("        ├── openai/          # provider module")
print("        │   ├── module.yaml       # Module metadata")
print("        │   ├── provider.py       # Implementation")
print("        │   └── config.schema.json  # Optional config schema")
print("        ├── web-search/      # tool module")
print("        │   ├── module.yaml")
print("        │   └── tool.py")
print("        └── logging/         # hook module")
print("            ├── module.yaml")
print("            └── hook.py")
print("\nNot: collections/my-collection/modules/providers/openai/")
print("But: modules/my-collection/openai/")
print("\nModule ID: my-collection/openai")
print("\nmodule.yaml format:")
print("---")
print("name: OpenAI Provider")
print("type: provider")
print("version: '1.0.0'")
print("description: 'LLM provider for OpenAI'")
print("entry_point: provider.py")
print("config_schema:")
print("  model:")
print("    type: string")
print("    default: gpt-4")
print("\nKey Architectural Points:")
print("  • Flat structure: modules/{collection}/{name}/")
print("  • Collection name provides namespace isolation")
print("  • Module type determined from module.yaml")
print("  • Follows Unix FHS pattern (like /usr/share/)")
print("  • User-centric: 'show me providers' not 'show me collections'")

## Summary

### Module Discovery

- ✓ List all modules across all types from flat structure
- ✓ List modules filtered by type (providers, tools, hooks, orchestrators)
- ✓ Get detailed module information
- ✓ Understand module IDs and flat directory structure

### Module ID Format

Module IDs follow the pattern: `{collection}/{name}`

Examples:
- `base-collection/openai`
- `my-tools/web-search`
- `base-collection/logging`

**Collection name provides namespace isolation** - prevents conflicts when multiple collections provide modules with the same name.

### Module Discovery Process

1. Scans `{AMPLIFIERD_ROOT}/local/share/modules/**/*.yaml` recursively
2. Detects collection from directory path
3. Reads metadata from `module.yaml` files (including type)
4. Returns module info with collection, type, and location

### Flat Directory Structure

**Not** (nested in collections):
```
collections/
└── my-collection/
    └── modules/
        └── providers/
            └── openai/
```

**But** (flat by collection):
```
modules/
└── my-collection/          # Collection namespace
    ├── openai/             # provider module
    │   └── module.yaml     # type: provider
    ├── web-search/         # tool module
    │   └── module.yaml     # type: tool
    └── logging/            # hook module
        └── module.yaml     # type: hook
```

**Benefits**:
- **Simpler structure**: Fewer nested directories
- **Module type in metadata**: Explicit in module.yaml
- **User-centric**: Focus on what users want, not package structure
- **Namespace isolation**: Collection name prevents naming conflicts
- **Unix precedent**: Like `/usr/share/{package}/` structure
- **Separation of concerns**: Package management separate from resource discovery

### Module Structure

```
modules/{collection}/{name}/
├── module.yaml       # Required metadata file (includes type)
├── {entry_point}     # Implementation (e.g., provider.py)
└── config.schema.json  # Optional configuration schema
```

### API Endpoints Reference

| Method | Endpoint | Description |
|--------|----------|-------------|
| GET | `/api/v1/modules/` | List all modules |
| GET | `/api/v1/modules/?type={type}` | Filter modules by type |
| GET | `/api/v1/modules/providers` | List provider modules |
| GET | `/api/v1/modules/tools` | List tool modules |
| GET | `/api/v1/modules/hooks` | List hook modules |
| GET | `/api/v1/modules/orchestrators` | List orchestrator modules |
| GET | `/api/v1/modules/{id}` | Get module details |

### Key Architectural Points

- **Flat structure**: Modules organized by collection, not by type subdirectory
- **Simple discovery**: Single recursive scan of `modules/**/*.yaml`
- **Clear structure**: Predictable path conventions
- **Metadata driven**: `module.yaml` defines module properties including type
- **Namespace isolation**: Collection name preserved to prevent conflicts
- **User-centric**: "Show me providers" not "show me collections with providers"

### Comparison: Nested vs. Flat

**Traditional nested approach**:
- Structure: `collections/*/modules/providers/*/`
- User asks: "Show me providers"
- System: Scans `collections/*/modules/providers/`
- Focus: Collections that happen to have providers

**Flat approach**:
- Structure: `modules/*/*/module.yaml`
- User asks: "Show me providers"
- System: Scans `modules/**/module.yaml` and filters by type
- Focus: All providers from all sources

**The flat approach is simpler and more intuitive for users.**

## Updated Summary: Profile-Driven Module System

### New Capabilities

- ✓ **Profile declares module dependencies**: `modules.sources` in profile YAML
- ✓ **Automatic module installation**: Sync endpoint resolves and installs modules
- ✓ **Content-addressable caching**: Git commit hashes enable deduplication
- ✓ **Symlink-based versioning**: Multiple versions coexist, switch with symlinks
- ✓ **Module state cache**: `module_state.json` tracks installation status
- ✓ **Version management**: List, switch, and cleanup module versions
- ✓ **Integration with profiles**: Profiles fully manage their module dependencies

### Module Resolution Workflow

```
Profile Sync Requested
         ↓
Parse profile.modules.sources
         ↓
For each source:
  ├─ Clone/pull repository
  ├─ Compute git commit hash
  ├─ Extract modules to state/modules/{hash}/
  ├─ Create symlinks from modules/{collection}/ to state
  └─ Update module_state.json
         ↓
Modules ready for use
```

### Directory Structure After Installation

```
modules/
├── my-collection/           # Active version (symlinks)
│   ├── openai/ -> ../state/modules/abc123def456/openai/
│   ├── web-search/ -> ../state/modules/abc123def456/web-search/
│   └── logging/ -> ../state/modules/abc123def456/logging/
├── another-collection/
└── profiles/{collection}/   # Also included if present
    └── some-profile.yaml

state/
└── modules/
    ├── abc123def456/        # Version 1 (git hash)
    │   ├── openai/
    │   │   └── module.yaml  # type: provider
    │   ├── web-search/
    │   │   └── module.yaml  # type: tool
    │   └── logging/
    │       └── module.yaml  # type: hook
    └── xyz789uvw012/        # Version 2 (git hash)
        └── openai/
            └── module.yaml
```

### Key Advantages

- **Declarative**: Profiles describe their complete dependency tree
- **Automatic**: No manual module installation needed
- **Efficient**: Content-addressable storage prevents duplication
- **Flexible**: Multiple versions manageable side-by-side
- **Clean**: Easy cleanup of unused versions
- **Observable**: State cache provides auditability

### API Endpoints Reference (Updated)

| Method | Endpoint | Description |
|--------|----------|-------------|
| GET | `/api/v1/modules/` | List all modules |
| GET | `/api/v1/modules/{type}` | List modules by type |
| GET | `/api/v1/modules/{id}` | Get module details |
| POST | `/api/v1/profiles/sync-modules` | Sync active profile's modules |
| GET | `/api/v1/modules/cache-stats` | Get cache usage statistics |
| GET | `/api/v1/modules/cache-versions` | List cached versions |
| POST | `/api/v1/modules/{id}/activate` | Activate specific version |
| DELETE | `/api/v1/modules/cache/{hash}` | Remove cached version |

In [ ]:
# Get module cache statistics
response = requests.get(f"{API_BASE}/modules/cache-stats")
cache_stats = print_response(response, "MODULE CACHE STATISTICS")

if cache_stats and cache_stats.get("versions"):
    print("\n✓ Module cache overview:")
    print(f"  Total size: {cache_stats.get('totalCacheSize', 'N/A')}")
    print(f"  Cached versions: {len(cache_stats['versions'])}")

    print("\nCached module versions:")
    for version in cache_stats["versions"][:5]:
        active = "[ACTIVE]" if version.get("active") else ""
        print(f"  - {version['cacheKey'][:16]}... {active}")
        print(f"    Size: {version.get('size', 'N/A')}")
        print(f"    Modules: {version.get('modules', 0)}")
else:
    print("No module cache information available.")

In [ ]:
## Example: Demonstrate Profile-Driven Module Installation

# First, get the active profile
response = requests.get(f"{API_BASE}/profiles/active")
active_profile = response.json() if response.ok else None

if active_profile:
    print(f"Active Profile: {active_profile['name']}")
    print("\nModule sources in profile:")

    # Show declared module sources
    if "moduleSources" in active_profile:
        for source in active_profile["moduleSources"]:
            print(f"\n  Source: {source.get('url', 'N/A')}")
            print(f"  Branch: {source.get('branch', 'main')}")
            print(f"  Modules: {len(source.get('modules', []))}")
            for module in source.get("modules", [])[:3]:
                print(f"    - {module}")
    else:
        print("  (No module sources declared)")

    # Show currently used modules
    print("\nModules referenced by profile:")
    providers = active_profile.get("providers", [])
    tools = active_profile.get("tools", [])
    hooks = active_profile.get("hooks", [])

    print(f"  Providers: {len(providers)}")
    for provider in providers[:3]:
        print(f"    - {provider.get('module', 'N/A')}")

    print(f"  Tools: {len(tools)}")
    for tool in tools[:3]:
        print(f"    - {tool.get('module', 'N/A')}")

    print(f"  Hooks: {len(hooks)}")
    for hook in hooks[:3]:
        print(f"    - {hook.get('module', 'N/A')}")
else:
    print("No active profile. Activate a profile to see module sources.")

## Profile-Driven Module Installation

Modules are automatically installed and resolved when profiles that reference them are synced. This enables profiles to manage their complete dependency tree.

### Module Sources in Profiles

Profiles declare module sources and specific modules needed:

```yaml
profile:
  name: dev-profile
  version: "1.0.0"
  description: "Development environment with AI providers and tools"

modules:
  sources:
    # Source 1: Base collection
    - url: https://github.com/amplifier/base-modules.git
      branch: main
      directory: modules/
      modules:
        - base/openai
        - base/anthropic
        - base/web-search
        - base/logging

    # Source 2: Custom organization collection
    - url: https://github.com/myorg/custom-modules.git
      branch: main
      directory: modules/
      modules:
        - myorg/internal-llm
        - myorg/internal-api

providers:
  - module: base/openai
    config:
      model: gpt-4
      temperature: 0.7

  - module: myorg/internal-llm
    config:
      endpoint: "${env:INTERNAL_LLM_ENDPOINT}"

tools:
  - module: base/web-search
    config:
      timeout: 30

hooks:
  - module: base/logging
    config:
      level: debug
```

### Sync Modules Endpoint

Sync all modules required by the active profile:

```python
# Trigger module sync for active profile
response = requests.post(f"{API_BASE}/profiles/sync-modules")

# Successful response
{
  "status": "success",
  "profileName": "dev-profile",
  "modulesSynced": 5,
  "timestamp": "2025-11-21T10:30:00Z",
  "modules": [
    {
      "id": "base/openai",
      "status": "installed",
      "cacheKey": "abc123def456",
      "path": "/path/to/.../modules/base/openai",
      "installed": true
    },
    {
      "id": "myorg/internal-llm",
      "status": "installed",
      "cacheKey": "xyz789uvw012",
      "path": "/path/to/.../modules/myorg/internal-llm",
      "installed": true
    }
  ]
}
```

### Content-Addressable Module Caching

Modules are cached by git commit hash to enable deduplication:

```
{AMPLIFIERD_ROOT}/local/share/modules/
├── base/  # Active version (symlinks)
│   ├── openai/ -> ../../state/modules/abc123def456/openai/
│   ├── anthropic/ -> ../../state/modules/abc123def456/anthropic/
│   ├── web-search/ -> ../../state/modules/abc123def456/web-search/
│   └── logging/ -> ../../state/modules/abc123def456/logging/
│
└── myorg/  # Active version (symlinks)
    ├── internal-llm/ -> ../../state/modules/xyz789uvw012/internal-llm/
    └── internal-api/ -> ../../state/modules/xyz789uvw012/internal-api/

{AMPLIFIERD_ROOT}/local/state/modules/
├── abc123def456/  # Git commit hash for version 1
│   ├── openai/
│   │   └── module.yaml  # type: provider
│   ├── anthropic/
│   │   └── module.yaml  # type: provider
│   ├── web-search/
│   │   └── module.yaml  # type: tool
│   └── logging/
│       └── module.yaml  # type: hook
└── xyz789uvw012/  # Git commit hash for version 2
    ├── internal-llm/
    │   └── module.yaml  # type: provider
    └── internal-api/
        └── module.yaml  # type: tool
```

### Module State Cache

Installation state is cached for quick lookup and version tracking:

```python
# Cache structure: {AMPLIFIERD_ROOT}/.cache/module_state.json

{
  "profile": "dev-profile",
  "lastSync": "2025-11-21T10:30:00Z",
  "modules": {
    "base/openai": {
      "version": "1.0.0",
      "cacheKey": "abc123def456",
      "installed": true,
      "path": "/path/to/.../modules/base/openai",
      "symlinkTarget": "/path/to/.../state/modules/abc123def456/openai"
    },
    "myorg/internal-llm": {
      "version": "1.2.3",
      "cacheKey": "xyz789uvw012",
      "installed": true,
      "path": "/path/to/.../modules/myorg/internal-llm",
      "symlinkTarget": "/path/to/.../state/modules/xyz789uvw012/internal-llm"
    }
  },
  "cacheVersions": {
    "abc123def456": {
      "source": "https://github.com/amplifier/base-modules.git",
      "branch": "main",
      "timestamp": "2025-11-21T10:00:00Z",
      "modules": [
        "base/openai",
        "base/anthropic",
        "base/web-search",
        "base/logging"
      ]
    }
  }
}
```

### Symlink Structure for Deduplication

Module symlinks enable efficient version management:

```python
# Check which cache version is active
import os

module_path = Path("{AMPLIFIERD_ROOT}/local/share/modules/base/openai")

# Resolve symlink to cache version
cache_path = module_path.resolve()  # Points to state/modules/abc123def456/...
git_hash = cache_path.parts[-2]  # Extract hash from path

print(f"Active: {module_path}")
print(f"Cache: {cache_path}")
print(f"Hash: {git_hash}")
```

### Module Installation Workflow

Complete flow from profile declaration to module availability:

```
1. Profile specifies module sources
   ↓
2. Profile sync triggered (manual or automatic)
   ↓
3. Resolve module declarations
   ↓
4. For each module source:
   a. Clone/pull repository
   b. Compute git commit hash
   c. Extract modules to state/modules/{hash}/
   d. Create symlinks from modules/{collection}/ to state
   e. Update module_state.json
   ↓
5. Modules available for use
   ↓
6. Profile uses modules via references
```

### Version Management and Updates

Multiple versions coexist in cache with easy switching:

```python
# List available module versions
response = requests.get(f"{API_BASE}/modules/cache-versions")

{
  "modules": {
    "base/openai": [
      {
        "version": "1.0.0",
        "cacheKey": "abc123def456",
        "installed": true,
        "active": true,
        "source": "https://github.com/amplifier/base-modules.git",
        "timestamp": "2025-11-21T10:00:00Z"
      },
      {
        "version": "0.9.0",
        "cacheKey": "older123hash",
        "installed": true,
        "active": false,
        "source": "https://github.com/amplifier/base-modules.git",
        "timestamp": "2025-11-15T09:00:00Z"
      }
    ]
  }
}

# Activate specific module version
response = requests.post(
    f"{API_BASE}/modules/base/openai/activate",
    json={"cacheKey": "older123hash"}
)
```

### Cleanup and Cache Management

Unused module versions can be removed from cache:

```python
# Get cache usage statistics
response = requests.get(f"{API_BASE}/modules/cache-stats")

{
  "totalCacheSize": "2.3 GB",
  "versions": [
    {
      "cacheKey": "abc123def456",
      "size": "1.2 GB",
      "modules": 8,
      "active": true,
      "lastAccessed": "2025-11-21T10:30:00Z"
    },
    {
      "cacheKey": "older123hash",
      "size": "850 MB",
      "modules": 6,
      "active": false,
      "lastAccessed": "2025-11-15T09:00:00Z"
    }
  ]
}

# Remove unused cache version
response = requests.delete(f"{API_BASE}/modules/cache/{old_hash}")
result = print_response(response, "CLEAN UP OLD CACHE VERSION")
```